In [1]:
%load_ext autoreload
%autoreload 2

from PIL import Image
import napari
import numpy as np
import os

from scribbles_creator import *
from scribbles_testing.cellpose_data_handler import *

## Define where the images are located

In [2]:
folder_path = "../cellpose_train_imgs/"

## Create ground truth for pixel classification from image mask (instance segmentation)

In [ ]:
# for img_num in range(0, 540):
#     create_cellpose_gt(folder_path, img_num, save_res=True, show_res=False)

## Create scribble annotation from an image mask of the cell pose dataset

Define scribbles parameters

In [27]:
# Which scribbles to create
mode = "all" # "lines" "prim_sk" "sec_sk"
class_dist = "balanced" # 0.1 "absolute" "relative"
rel_scribble_len = False #10 #100
enforce_max_perc = True
bins = [1] #[0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1, 2.5, 5, 10]
scribble_width = 1
# bin_width = [(0.01, 1), (0.025, 1), (0.05, 1), (0.1, 1),
            #  (0.25, 2), (0.5, 2), (1, 2),
            #  (2.5, 4), (5, 4), (10, 4)]
margin = 0.75

# all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
# suff = all_suff[:1]
s = "a"
scribbles_seed = 1

# What to do
save_res = False
show_res = True
print_steps = True

Loop and create scribbles

In [28]:
percentages = {}
for bin in bins:
# for bin, scribble_width in bin_width:
    percentages[bin] = []
    for img_num in range(0, 540, 300): #[200, 50, 272, 351, 493]:#, 383, 419]:#range(0, 10):
        print(f"IMG {img_num}: {bin}, {s}")
        np.random.seed(scribbles_seed)
        scribbles, perc_labelled = create_cellpose_scribble(folder_path, img_num, bin=bin, margin=margin, rel_scribble_len=rel_scribble_len, scribble_width=scribble_width, mode=mode, 
                                                            class_dist=class_dist, enforce_max_perc=enforce_max_perc,
                                                            save_res=save_res, suff=s, show_res=show_res, print_steps=print_steps)
        percentages[bin].append(perc_labelled)
        print("\n")

IMG 0: 1, a

max. perc.: 1, margin: 0.75, rel_scribble_len: 20.00, width: 1, mode: all, class_dist: balanced, enforce_max_perc: True, print_steps: True

CLASS 1, max. pixel: 0.983% = 998 pixels
   sk_max_pix: 332.00, sq_size: 22, sq_pix_range: (11, 44)
      prim_sk_squares pix: 328 = 0.323%
      sec_sk_squares pix: 330 = 0.325%
   lines_max_pix: 332.00, line_pix_range: (11, 44)
      lines pix: 327 = 0.322%
CLASS 1 pixels: 983 = 0.968% 

CLASS 2, max. pixel: 1.019% = 962 pixels
   sk_max_pix: 320.00, sq_size: 22, sq_pix_range: (11, 44)
      prim_sk_squares pix: 311 = 0.329%
      sec_sk_squares pix: 320 = 0.339%
   lines_max_pix: 320.00, line_pix_range: (11, 44)
      lines pix: 310 = 0.328%
CLASS 2 pixels: 940 = 0.995% 

TOTAL annotation: 1923 = 0.981% 



IMG 300: 1, a

max. perc.: 1, margin: 0.75, rel_scribble_len: 20.00, width: 1, mode: all, class_dist: balanced, enforce_max_perc: True, print_steps: True

CLASS 1, max. pixel: 2.139% = 855 pixels
   sk_max_pix: 285.00, sq_size: 2

Quick report of labelling percentages:

In [ ]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

Check percentages of saved scribbles

In [ ]:
perc_dict = {}
for bin in bins:
    bin_perc_list = []
    for img_num in range(0,540,40):
        scrib = np.array(Image.open(f"{folder_path}{img_num:03d}_scribbles_{mode}_{bin_for_file(bin)}_{s}.png"))
        perc_labelled = np.sum(scrib > 0) / scrib.size
        bin_perc_list.append(perc_labelled)
    perc_dict[bin] = bin_perc_list

Show the last created scribbles

In [ ]:
img_data = get_cellpose_img_data(folder_path, img_num, load_img=True, load_gt=True, load_scribbles=True, mode=mode, bin=bin, scribble_width=scribble_width, suff=s)
img = img_data["img"]
ground_truth = img_data["gt"]

In [ ]:
v = napari.Viewer()
v.add_image(img)
v.add_labels(ground_truth)
v.add_labels(scribbles)

## Load and show scribbles and ground truth from saved files

In [ ]:
# Show the image, ground truth and the scribble annotation in a napari viewer
img_num = 0
mode = "all"
bin = 0.1
suff = "a"
img_data = get_cellpose_img_data(folder_path, img_num, load_img=True, load_gt=True, load_scribbles=True, mode=mode, bin=bin, suff=suff)
v = napari.Viewer()
v.add_image(img_data["img"], name="image")
v.add_labels(img_data["gt"], name="ground truth")
v.add_labels(img_data["scribbles"], name="scribbles")

## When we have performed segmentation in the Napari plugin, we can calculate its accuracy

In [ ]:
# segmentation = v.layers['segmentation'].data
# acc = np.round(100*np.sum(segmentation == ground_truth) / np.sum(ground_truth > 0), 2)
# print(f"Accuracy (pixels correctly segmented): {acc}%")